In [ ]:
import pygame

from pygame.locals import (
    K_RIGHT,
    KEYDOWN,
    QUIT,
)

In [ ]:
import numpy as np

from customer_model import Customer, CustomerStatus
from taxi_model import Taxi, TaxiStatus

INITIAL_FEE = 5
LENGTH_FEE = 2
N_TAXIS = 3
N_CUSTOMERS = 5


class TaxiService:
    def __init__(self, city_plan):
        self.customers: list[Customer] = []
        self.city_plan: dict[str, list[str]] = city_plan
        self.taxis: dict[str, Taxi] = {}
        self.taxis_in_vertices = {k: [] for k in city_plan.keys()}
        self.new_taxi_key: int = 1

        for i in range(N_TAXIS):
            self.generate_new_taxi()

        for i in range(N_CUSTOMERS):
            self._generate_new_customer()

    def assign_taxi_to_customer(self):
        for customer in self.customers:
            if customer.status != CustomerStatus.NO_TAXI:
                continue
            taxi_id, pickup_path = self.find_closest_taxi(customer.current_vertex)
            if taxi_id:
                customer.assigned_taxi = taxi_id
                customer.pickup_path = pickup_path
                customer.status = CustomerStatus.WAITING
                self.taxis[taxi_id].status = TaxiStatus.GOING_TO_CUSTOMER
                self.taxis[taxi_id].n_customers_delivered += 1
                self.taxis[taxi_id].total_income += INITIAL_FEE
            else:
                break

    @staticmethod
    def _create_path(current_vertex, predecessors):
        path = []
        vertex = predecessors[current_vertex]

        while vertex:
            path.append(vertex)
            vertex = predecessors[vertex]

        return path

    def find_closest_taxi(self, customer_vertex):
        predecessors = {customer_vertex: None}
        visited = {customer_vertex}
        queue = [customer_vertex]

        while queue:
            processed_vertex = queue.pop(0)
            for taxi in self.taxis_in_vertices[processed_vertex]:
                if self.taxis[taxi].status == TaxiStatus.FREE:
                    path = self._create_path(processed_vertex, predecessors)
                    return taxi, path

            for neighbour_vertex in self.city_plan[processed_vertex]:
                if neighbour_vertex not in visited:
                    visited.add(neighbour_vertex)
                    queue.append(neighbour_vertex)
                    predecessors[neighbour_vertex] = processed_vertex
        return None, None

    def find_destination_path(self, customer_vertex, destination_vertex):
        predecessors = {destination_vertex: None}
        visited = {destination_vertex}
        queue = [destination_vertex]

        while queue:
            processed_vertex = queue.pop(0)
            if processed_vertex == customer_vertex:
                path = self._create_path(processed_vertex, predecessors)
                return path

            for neighbour_vertex in self.city_plan[processed_vertex]:
                if neighbour_vertex not in visited:
                    visited.add(neighbour_vertex)
                    queue.append(neighbour_vertex)
                    predecessors[neighbour_vertex] = processed_vertex

        return None

    def generate_new_taxi(self):
        possible_lst = list(self.city_plan.keys())
        current_vertex = np.random.choice(possible_lst)

        new_key = str(self.new_taxi_key)
        self.new_taxi_key += 1

        self.taxis[new_key] = Taxi(current_vertex)
        self.taxis_in_vertices[current_vertex].append(new_key)

    def _generate_new_customer(self):
        possible_lst = list(self.city_plan.keys())
        current_vertex = np.random.choice(possible_lst)
        possible_lst.remove(current_vertex)
        destination_vertex = np.random.choice(possible_lst)

        customer = Customer(current_vertex, destination_vertex)
        customer.destination_path = self.find_destination_path(current_vertex, destination_vertex)

        self.customers.append(customer)

    def generate_new_customers(self):
        customer_proba = [0.99, 0.005, 0.005]
        n_new_customers = np.random.choice(list(range(len(customer_proba))), p=customer_proba)

        for _ in range(n_new_customers):
            self._generate_new_customer()

    def _process_customer_waiting(self, customer):
        if customer.pickup_path:
            current_vertex = self.taxis[customer.assigned_taxi].current_vertex
            new_vertex = customer.pickup_path.pop(0)

            self.taxis_in_vertices[current_vertex].remove(customer.assigned_taxi)
            self.taxis_in_vertices[new_vertex].append(customer.assigned_taxi)

            self.taxis[customer.assigned_taxi].current_vertex = new_vertex
            self.taxis[customer.assigned_taxi].total_distance += 1

        else:
            customer.status = CustomerStatus.INSIDE

    def _process_customer_inside(self, customer):
        if customer.destination_path:
            current_vertex = self.taxis[customer.assigned_taxi].current_vertex
            new_vertex = customer.destination_path.pop(0)

            self.taxis_in_vertices[current_vertex].remove(customer.assigned_taxi)
            self.taxis_in_vertices[new_vertex].append(customer.assigned_taxi)

            customer.current_vertex = new_vertex
            self.taxis[customer.assigned_taxi].current_vertex = new_vertex
            self.taxis[customer.assigned_taxi].total_distance += 1
            self.taxis[customer.assigned_taxi].total_income += LENGTH_FEE

        else:
            customer.status = CustomerStatus.END

    def make_step(self):
        self.generate_new_customers()
        self.assign_taxi_to_customer()

        customers_to_delete = []

        for i, customer in enumerate(self.customers):
            if customer.status == CustomerStatus.WAITING:
                self._process_customer_waiting(customer)
            elif customer.status == CustomerStatus.INSIDE:
                self._process_customer_inside(customer)
            elif customer.status == CustomerStatus.END:
                self.taxis[customer.assigned_taxi].status = TaxiStatus.FREE
                customers_to_delete.append(i)

        for i in customers_to_delete[::-1]:
            del self.customers[i]


In [ ]:
v_corr = {}
g = {}
cnt = 0


def add_line_horizontal(line_start, line_end, position, block_size=5):
    for i in range(line_start, line_end, block_size):
        v_corr[f"{i} {position}"] = (i, position)
        g[f"{i} {position}"] = (
            [f"{i-20} {position}", f"{i} {position-20}", f"{i+20} {position}", f"{i} {position+20}"]
        )


def add_line_vertical(line_start, line_end, position, block_size=5):
    for i in range(line_start, line_end, block_size):
        v_corr[f"{position} {i}"] = (position, i)
        g[f"{position} {i}"] = (
            [f"{position-20} {i}", f"{position} {i-20}", f"{position+20} {i}", f"{position} {i+20}"]
        )


In [ ]:
# # Horizontal:
# add_line_horizontal(260, 425, 465)  # 262, 466 - 427, 466 
# add_line_horizontal(640, 835, 465)  # 642, 466 - 836, 466
# add_line_horizontal(260, 835, 485)  # 262, 484 - 837, 484
# add_line_horizontal(260, 835, 485)  # 262, 502 - 642, 502 
# add_line_horizontal(640, 835, 515)  # 642, 515 - 836, 515
# # add_line_horizontal(260, 835, 520)  # 262, 522 - 836, 522
# add_line_horizontal(640, 835, 540)  # 642, 543 - 836, 543
# add_line_horizontal(260, 835, 560)  # 642, 562 - 836, 562
# add_line_horizontal(260, 355, 540)  # 262, 540 - 355, 540
# add_line_horizontal(260, 355, 560)  # 262, 560 - 355, 560
# add_line_horizontal(260, 635, 580)  # 262, 580 - 637, 580
# add_line_horizontal(260, 780, 580)  # 262, 592 - 782, 592
# add_line_horizontal(260, 725, 600)  # 262, 603 - 725, 603 
# add_line_horizontal(260, 745, 615)  # 262, 613 - 745, 613
# add_line_horizontal(260, 725, 625)  # 262, 624 - 725, 624
# add_line_horizontal(260, 725, 640)  # 262, 638 - 725, 638
# add_line_horizontal(260, 725, 655)  # 262, 654 - 725, 654

In [ ]:
# add_line_vertical(450, 700, 270)
# add_line_vertical(450, 700, 700)

In [ ]:
add_line_horizontal(20, 620, 20)
add_line_horizontal(20, 620, 100)
add_line_horizontal(20, 620, 200)
add_line_horizontal(20, 160, 320)
add_line_horizontal(160, 640, 420)

add_line_vertical(20, 320, 20)
add_line_vertical(20, 420, 160)
add_line_vertical(20, 420, 400)
add_line_vertical(20, 420, 620)

In [ ]:
for x in g:
    lst_to_remove = []
    for j in range(4):
        if g[x][j] not in g:
            lst_to_remove.append(j)

    for j in lst_to_remove[::-1]:
        del g[x][j]

In [ ]:
pygame.init()
# Parameters
w_width = 900
w_height = 900
# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])

# Set background
screen.fill((255, 255, 255))

ts = TaxiService(g)


def draw(screen, v_cor):
    block_size = 5

    for i, j in v_cor.values():
        rect = pygame.Rect(i, j, block_size - 1, block_size - 1)
        pygame.draw.rect(screen, (210, 210, 210), rect, 0)

    for taxi in ts.taxis.values():
        i, j = v_corr[taxi.current_vertex]
        rect = pygame.Rect(i, j, block_size - 3, block_size - 3)
        pygame.draw.rect(screen, (210, 150, 10), rect, 0)

    for customer in ts.customers:
        i, j = v_corr[customer.current_vertex]
        rect = pygame.Rect(i + 2, j + 2, block_size - 3, block_size - 3)
        pygame.draw.rect(screen, (100, 150, 10), rect, 0)

    pygame.display.flip()


draw(screen, v_corr)
running = True

time_delay = 200  # 0.2 s
timer_event = pygame.USEREVENT + 1
pygame.time.set_timer(timer_event, time_delay)

while running:
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False

        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                ts.make_step()
                draw(screen, v_corr)
    
pygame.quit()

In [ ]:
ts.customers[0].status

In [ ]:
ts.customers[1].status

In [ ]:
ts.taxis["1"].__hash__()